In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
# Setting logging level
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

Loading the data of the [SLAC dataset](http://jmir.sourceforge.net/Codaich.html), with 250 MIDI files. The dataset is equally balanced over the classes.

The vectors are computed using [midi2vec](https://github.com/midi-ld/midi2vec).

The dataset provides 2 kind of classes (genres and subgenres), respectively of 5 and 10 elements:

- Blues
    - Modern Blues
    - Traditional Blues
- Classical
    - Baroque
    - Romantic
- Jazz
    - Bop
    - Swing
- Rap
    - Hardcore Rap
    - Pop Rap
- Rock
    - Alternative Rock
    - Metal

In [3]:
data = np.loadtxt('./dataset-slac/vectors.txt')
genres = np.loadtxt('./dataset-slac/genre.txt', dtype=np.str, delimiter=',')
subgenres = np.loadtxt('./dataset-slac/subgenre.txt', dtype=np.str, delimiter=',')

The labels are represented with the 1-hot encoding

In [4]:
encoder = LabelBinarizer()
def onehot_labels(labels):
    return encoder.fit_transform(labels)
    
def onehot2str(onehot):
    return encoder.inverse_transform(np.array([onehot]))[0]

### Experiment 1: Genres 

In [5]:
labels_1hot = onehot_labels(genres)
    
onehot2str(labels_1hot[0])

'Classical'

In [6]:
data.shape

(249, 100)

In [7]:
labels_1hot.shape

(249, 5)

In [8]:
num_input = data.shape[1]
num_output = labels_1hot.shape[1]

num_input, num_output

(100, 5)

We use a Neural Network with 3 dense layers, trained with the Adam Optimizer.

In [9]:
def create_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Dense(100, activation='relu', input_shape=(num_input,)))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(num_output, activation='sigmoid'))

    model.compile(optimizer=tf.train.AdamOptimizer(0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [10]:
neural_network = KerasClassifier(build_fn=create_model, 
                                 epochs=100, 
                                 batch_size=32)

The network is evaluated with 10-fold cross-validation. The final score is the average of the results.

In [11]:
scores = cross_val_score(neural_network, data, labels_1hot, cv=10, verbose=0)
scores

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
224/224 [==============================] - 0s 682us/sample - loss: 0.4983 - acc: 0.7589
Epoch 2/100
224/224 [==============================] - 0s 51us/sample - loss: 0.2527 - acc: 0.9071
Epoch 3/100
224/224 [==============================] - 0s 48us/sample - loss: 0.1268 - acc: 0.9554
Epoch 4/100
224/224 [==============================] - 0s 48us/sample - loss: 0.0575 - acc: 0.9804
Epoch 5/100
224/224 [==============================] - 0s 51us/sample - loss: 0.0206 - acc: 0.9982
Epoch 6/100
224/224 [==============================] - 0s 44us/sample - loss: 0.0115 - acc: 0.9982
Epoch 7/100
224/224 [==============================] - 0s 47us/sample - loss: 0.0056 - acc: 0.9991
Epoch 8/100
224/224 [==============================] - 0s 44us/sample - loss: 0.0036 - acc: 0.9991
Epoch 9/100
224/224 [==============================] - 0s 48us/sample - loss: 0.0023 - 

224/224 [==============================] - 0s 45us/sample - loss: 4.2519e-05 - acc: 1.0000
Epoch 53/100
224/224 [==============================] - 0s 52us/sample - loss: 4.1066e-05 - acc: 1.0000
Epoch 54/100
224/224 [==============================] - 0s 50us/sample - loss: 4.0039e-05 - acc: 1.0000
Epoch 55/100
224/224 [==============================] - 0s 44us/sample - loss: 3.8551e-05 - acc: 1.0000
Epoch 56/100
224/224 [==============================] - 0s 49us/sample - loss: 3.7243e-05 - acc: 1.0000
Epoch 57/100
224/224 [==============================] - 0s 47us/sample - loss: 3.6280e-05 - acc: 1.0000
Epoch 58/100
224/224 [==============================] - 0s 52us/sample - loss: 3.5158e-05 - acc: 1.0000
Epoch 59/100
224/224 [==============================] - 0s 45us/sample - loss: 3.4184e-05 - acc: 1.0000
Epoch 60/100
224/224 [==============================] - 0s 51us/sample - loss: 3.3048e-05 - acc: 1.0000
Epoch 61/100
224/224 [==============================] - 0s 47us/sample - loss

224/224 [==============================] - 0s 64us/sample - loss: 0.0058 - acc: 0.9991
Epoch 9/100
224/224 [==============================] - 0s 59us/sample - loss: 0.0036 - acc: 0.9991
Epoch 10/100
224/224 [==============================] - 0s 57us/sample - loss: 0.0023 - acc: 0.9991
Epoch 11/100
224/224 [==============================] - 0s 47us/sample - loss: 0.0015 - acc: 1.0000
Epoch 12/100
224/224 [==============================] - 0s 49us/sample - loss: 9.6345e-04 - acc: 1.0000
Epoch 13/100
224/224 [==============================] - 0s 52us/sample - loss: 7.2383e-04 - acc: 1.0000
Epoch 14/100
224/224 [==============================] - 0s 49us/sample - loss: 5.9858e-04 - acc: 1.0000
Epoch 15/100
224/224 [==============================] - 0s 44us/sample - loss: 4.5575e-04 - acc: 1.0000
Epoch 16/100
224/224 [==============================] - 0s 47us/sample - loss: 3.8229e-04 - acc: 1.0000
Epoch 17/100
224/224 [==============================] - 0s 50us/sample - loss: 3.3715e-04 - ac

224/224 [==============================] - 0s 53us/sample - loss: 1.5861e-05 - acc: 1.0000
Epoch 88/100
224/224 [==============================] - 0s 47us/sample - loss: 1.5507e-05 - acc: 1.0000
Epoch 89/100
224/224 [==============================] - 0s 47us/sample - loss: 1.5218e-05 - acc: 1.0000
Epoch 90/100
224/224 [==============================] - 0s 55us/sample - loss: 1.4934e-05 - acc: 1.0000
Epoch 91/100
224/224 [==============================] - 0s 52us/sample - loss: 1.4572e-05 - acc: 1.0000
Epoch 92/100
224/224 [==============================] - 0s 59us/sample - loss: 1.4328e-05 - acc: 1.0000
Epoch 93/100
224/224 [==============================] - 0s 64us/sample - loss: 1.4073e-05 - acc: 1.0000
Epoch 94/100
224/224 [==============================] - 0s 56us/sample - loss: 1.3813e-05 - acc: 1.0000
Epoch 95/100
224/224 [==============================] - 0s 53us/sample - loss: 1.3501e-05 - acc: 1.0000
Epoch 96/100
224/224 [==============================] - 0s 46us/sample - loss

224/224 [==============================] - 0s 50us/sample - loss: 5.8485e-05 - acc: 1.0000
Epoch 44/100
224/224 [==============================] - 0s 53us/sample - loss: 5.6297e-05 - acc: 1.0000
Epoch 45/100
224/224 [==============================] - 0s 53us/sample - loss: 5.3972e-05 - acc: 1.0000
Epoch 46/100
224/224 [==============================] - 0s 57us/sample - loss: 5.2231e-05 - acc: 1.0000
Epoch 47/100
224/224 [==============================] - 0s 54us/sample - loss: 5.0239e-05 - acc: 1.0000
Epoch 48/100
224/224 [==============================] - 0s 53us/sample - loss: 4.8298e-05 - acc: 1.0000
Epoch 49/100
224/224 [==============================] - 0s 53us/sample - loss: 4.6699e-05 - acc: 1.0000
Epoch 50/100
224/224 [==============================] - 0s 58us/sample - loss: 4.5155e-05 - acc: 1.0000
Epoch 51/100
224/224 [==============================] - 0s 60us/sample - loss: 4.3507e-05 - acc: 1.0000
Epoch 52/100
224/224 [==============================] - 0s 57us/sample - loss

224/224 [==============================] - 0s 51us/sample - loss: 1.0954e-05 - acc: 1.0000
Epoch 100/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.7374 - acc: 0.8640
Epoch 1/100
224/224 [==============================] - 0s 849us/sample - loss: 0.5139 - acc: 0.7625
Epoch 2/100
224/224 [==============================] - 0s 51us/sample - loss: 0.2284 - acc: 0.9250
Epoch 3/100
224/224 [==============================] - 0s 54us/sample - loss: 0.0955 - acc: 0.9705
Epoch 4/100
224/224 [==============================] - 0s 57us/sample - loss: 0.0513 - acc: 0.9857
Epoch 5/100
224/224 [==============================] - 0s 56us/sample - loss: 0.0258 - acc: 0.9938
Epoch 6/100
224/224 [==============================] - 0s 58us/sample - loss: 0.0128 - acc: 0.9973
Epoch 7/100
224/224 [==============================] - 0s 57us/sample - loss: 0.0070 - acc: 0.9982
Epoch 8/100
224/224 [==============================] - 0s 54us/sample - loss: 0.0044 - acc: 0.9991
Epoch 9/100
224/22

224/224 [==============================] - 0s 53us/sample - loss: 3.9769e-05 - acc: 1.0000
Epoch 56/100
224/224 [==============================] - 0s 54us/sample - loss: 3.8312e-05 - acc: 1.0000
Epoch 57/100
224/224 [==============================] - 0s 57us/sample - loss: 3.7314e-05 - acc: 1.0000
Epoch 58/100
224/224 [==============================] - 0s 61us/sample - loss: 3.6010e-05 - acc: 1.0000
Epoch 59/100
224/224 [==============================] - 0s 58us/sample - loss: 3.5082e-05 - acc: 1.0000
Epoch 60/100
224/224 [==============================] - 0s 55us/sample - loss: 3.4057e-05 - acc: 1.0000
Epoch 61/100
224/224 [==============================] - 0s 57us/sample - loss: 3.3209e-05 - acc: 1.0000
Epoch 62/100
224/224 [==============================] - 0s 55us/sample - loss: 3.2020e-05 - acc: 1.0000
Epoch 63/100
224/224 [==============================] - 0s 62us/sample - loss: 3.1109e-05 - acc: 1.0000
Epoch 64/100
224/224 [==============================] - 0s 58us/sample - loss

array([0.90400004, 0.88800001, 0.87199998, 0.92000002, 0.85600001,
       0.90399992, 0.86399996, 0.91199994, 0.99199998, 0.91666669])

In [12]:
np.mean(scores)

0.90286665558815

### Experiment 2: Subgenres 

The experiment is the same, but applied to the 10 subgenre classes.

In [13]:
labels_1hot = onehot_labels(subgenres)
    
onehot2str(labels_1hot[0])

'Romantic'

In [14]:
num_output = labels_1hot.shape[1]

num_input, num_output

(100, 10)

In [16]:
neural_network = KerasClassifier(build_fn=create_model, 
                                 epochs=100, 
                                 batch_size=32)

In [17]:
scores = cross_val_score(neural_network, data, labels_1hot, cv=10, verbose=0)
scores

Epoch 1/100
224/224 [==============================] - 0s 943us/sample - loss: 0.4127 - acc: 0.8388
Epoch 2/100
224/224 [==============================] - 0s 66us/sample - loss: 0.2434 - acc: 0.9134
Epoch 3/100
224/224 [==============================] - 0s 67us/sample - loss: 0.1596 - acc: 0.9313
Epoch 4/100
224/224 [==============================] - 0s 68us/sample - loss: 0.1028 - acc: 0.9634
Epoch 5/100
224/224 [==============================] - 0s 75us/sample - loss: 0.0680 - acc: 0.9759
Epoch 6/100
224/224 [==============================] - 0s 73us/sample - loss: 0.0403 - acc: 0.9893
Epoch 7/100
224/224 [==============================] - 0s 60us/sample - loss: 0.0243 - acc: 0.9942
Epoch 8/100
224/224 [==============================] - 0s 72us/sample - loss: 0.0172 - acc: 0.9955
Epoch 9/100
224/224 [==============================] - 0s 78us/sample - loss: 0.0116 - acc: 0.9973
Epoch 10/100
224/224 [==============================] - 0s 78us/sample - loss: 0.0073 - acc: 0.9987
Epoch 11

224/224 [==============================] - 0s 64us/sample - loss: 9.0117e-05 - acc: 1.0000
Epoch 61/100
224/224 [==============================] - 0s 69us/sample - loss: 8.8823e-05 - acc: 1.0000
Epoch 62/100
224/224 [==============================] - 0s 66us/sample - loss: 8.6118e-05 - acc: 1.0000
Epoch 63/100
224/224 [==============================] - 0s 70us/sample - loss: 8.0373e-05 - acc: 1.0000
Epoch 64/100
224/224 [==============================] - 0s 73us/sample - loss: 7.9783e-05 - acc: 1.0000
Epoch 65/100
224/224 [==============================] - 0s 69us/sample - loss: 7.6122e-05 - acc: 1.0000
Epoch 66/100
224/224 [==============================] - 0s 61us/sample - loss: 7.3749e-05 - acc: 1.0000
Epoch 67/100
224/224 [==============================] - 0s 69us/sample - loss: 7.1656e-05 - acc: 1.0000
Epoch 68/100
224/224 [==============================] - 0s 71us/sample - loss: 6.7862e-05 - acc: 1.0000
Epoch 69/100
224/224 [==============================] - 0s 68us/sample - loss

224/224 [==============================] - 0s 64us/sample - loss: 0.0011 - acc: 1.0000
Epoch 17/100
224/224 [==============================] - 0s 65us/sample - loss: 0.0010 - acc: 1.0000
Epoch 18/100
224/224 [==============================] - 0s 63us/sample - loss: 7.9379e-04 - acc: 1.0000
Epoch 19/100
224/224 [==============================] - 0s 64us/sample - loss: 7.2373e-04 - acc: 1.0000
Epoch 20/100
224/224 [==============================] - 0s 54us/sample - loss: 5.8876e-04 - acc: 1.0000
Epoch 21/100
224/224 [==============================] - 0s 59us/sample - loss: 5.5129e-04 - acc: 1.0000
Epoch 22/100
224/224 [==============================] - 0s 62us/sample - loss: 4.7457e-04 - acc: 1.0000
Epoch 23/100
224/224 [==============================] - 0s 64us/sample - loss: 4.4728e-04 - acc: 1.0000
Epoch 24/100
224/224 [==============================] - 0s 59us/sample - loss: 3.9785e-04 - acc: 1.0000
Epoch 25/100
224/224 [==============================] - 0s 59us/sample - loss: 3.6622

Epoch 95/100
224/224 [==============================] - 0s 61us/sample - loss: 2.6639e-05 - acc: 1.0000
Epoch 96/100
224/224 [==============================] - 0s 62us/sample - loss: 2.6086e-05 - acc: 1.0000
Epoch 97/100
224/224 [==============================] - 0s 65us/sample - loss: 2.5620e-05 - acc: 1.0000
Epoch 98/100
224/224 [==============================] - 0s 66us/sample - loss: 2.5050e-05 - acc: 1.0000
Epoch 99/100
224/224 [==============================] - 0s 66us/sample - loss: 2.4631e-05 - acc: 1.0000
Epoch 100/100
25/25 [==============================] - 0s 5ms/sample - loss: 0.6255 - acc: 0.9080
Epoch 1/100
224/224 [==============================] - 0s 1ms/sample - loss: 0.4212 - acc: 0.8388
Epoch 2/100
224/224 [==============================] - 0s 57us/sample - loss: 0.2432 - acc: 0.9089
Epoch 3/100
224/224 [==============================] - 0s 53us/sample - loss: 0.1597 - acc: 0.9362
Epoch 4/100
224/224 [==============================] - 0s 59us/sample - loss: 0.1032 -

Epoch 53/100
224/224 [==============================] - 0s 59us/sample - loss: 8.7310e-05 - acc: 1.0000
Epoch 54/100
224/224 [==============================] - 0s 60us/sample - loss: 8.4945e-05 - acc: 1.0000
Epoch 55/100
224/224 [==============================] - 0s 53us/sample - loss: 8.1545e-05 - acc: 1.0000
Epoch 56/100
224/224 [==============================] - 0s 51us/sample - loss: 7.9054e-05 - acc: 1.0000
Epoch 57/100
224/224 [==============================] - 0s 54us/sample - loss: 7.5343e-05 - acc: 1.0000
Epoch 58/100
224/224 [==============================] - 0s 53us/sample - loss: 7.1572e-05 - acc: 1.0000
Epoch 59/100
224/224 [==============================] - 0s 55us/sample - loss: 7.0465e-05 - acc: 1.0000
Epoch 60/100
224/224 [==============================] - 0s 53us/sample - loss: 6.7717e-05 - acc: 1.0000
Epoch 61/100
224/224 [==============================] - 0s 49us/sample - loss: 6.5408e-05 - acc: 1.0000
Epoch 62/100
224/224 [==============================] - 0s 58us/

224/224 [==============================] - 0s 65us/sample - loss: 0.0070 - acc: 0.9987
Epoch 11/100
224/224 [==============================] - 0s 65us/sample - loss: 0.0064 - acc: 0.9987
Epoch 12/100
224/224 [==============================] - 0s 63us/sample - loss: 0.0036 - acc: 0.9991
Epoch 13/100
224/224 [==============================] - 0s 67us/sample - loss: 0.0031 - acc: 0.9991
Epoch 14/100
224/224 [==============================] - 0s 63us/sample - loss: 0.0035 - acc: 0.9982
Epoch 15/100
224/224 [==============================] - 0s 63us/sample - loss: 0.0026 - acc: 0.9991
Epoch 16/100
224/224 [==============================] - 0s 62us/sample - loss: 0.0019 - acc: 0.9996
Epoch 17/100
224/224 [==============================] - 0s 62us/sample - loss: 0.0032 - acc: 0.9991
Epoch 18/100
224/224 [==============================] - 0s 61us/sample - loss: 0.0015 - acc: 0.9996
Epoch 19/100
224/224 [==============================] - 0s 62us/sample - loss: 0.0016 - acc: 0.9996
Epoch 20/100


224/224 [==============================] - 0s 72us/sample - loss: 7.5602e-05 - acc: 1.0000
Epoch 69/100
224/224 [==============================] - 0s 71us/sample - loss: 7.3961e-05 - acc: 1.0000
Epoch 70/100
224/224 [==============================] - 0s 62us/sample - loss: 7.2297e-05 - acc: 1.0000
Epoch 71/100
224/224 [==============================] - 0s 77us/sample - loss: 6.7710e-05 - acc: 1.0000
Epoch 72/100
224/224 [==============================] - 0s 72us/sample - loss: 6.9830e-05 - acc: 1.0000
Epoch 73/100
224/224 [==============================] - 0s 66us/sample - loss: 6.4494e-05 - acc: 1.0000
Epoch 74/100
224/224 [==============================] - 0s 64us/sample - loss: 6.5123e-05 - acc: 1.0000
Epoch 75/100
224/224 [==============================] - 0s 66us/sample - loss: 6.2860e-05 - acc: 1.0000
Epoch 76/100
224/224 [==============================] - 0s 63us/sample - loss: 5.7694e-05 - acc: 1.0000
Epoch 77/100
224/224 [==============================] - 0s 64us/sample - loss

array([0.89599997, 0.91199994, 0.88      , 0.90800005, 0.92400002,
       0.91600001, 0.90799993, 0.92399997, 0.96399993, 0.9375    ])

In [18]:
np.mean(scores)

0.9169499814510346